In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import keras
import tensorflow as tf
import random
import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
seed = 99
np.random.seed(seed)
random.seed(seed)

IMG_WIDTH = 224
IMG_HEIGHT = IMG_WIDTH
IMG_CHANNELS = 3

IMAGES_PATH = 'images/'
MASKS_PATH = 'GTmaps/''

In [ ]:
train_ratio = .8
test_ratio = .2

total_images = len(os.listdir(IMAGES_PATH))

train_size = int(total_images * train_ratio)
test_size = int(total_images * test_ratio)

In [ ]:
image_ids = os.listdir(IMAGES_PATH)

train_image_ids = random.sample(image_ids, train_size)
train_mask_ids = [id.replace('.png', '_GT.png') for id in train_image_ids]

image_ids = [id for id in image_ids if id not in train_image_ids]

test_image_ids = image_ids
test_mask_ids = [id.replace('.png', '_GT.png') for id in test_image_ids]

assert len(os.listdir(IMAGES_PATH)) == len(train_image_ids) + len(test_image_ids) == len(train_mask_ids) + len(test_mask_ids)

## Sanity check for images and masks

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(imread(IMAGES_PATH + train_image_ids[0])[:,:,:IMG_CHANNELS])
plt.subplot(1, 2, 2)
plt.imshow(imread(MASKS_PATH + train_mask_ids[0]))

In [ ]:
X_train = np.zeros((len(train_image_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_image_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

X_test = np.zeros((len(test_image_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_image_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

In [ ]:
def read_resized_images(path, ids, channels, data):
  for n, id_ in enumerate(ids):
    if (channels):
      image = imread(path + id_)[:,:,:channels]
      data[n] = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    else:
      image = imread(path + id_)
      data[n] = np.expand_dims(resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)

In [ ]:
read_resized_images(IMAGES_PATH, train_image_ids, IMG_CHANNELS, X_train)
read_resized_images(MASKS_PATH, train_mask_ids, None, Y_train)

In [ ]:
read_resized_images(IMAGES_PATH, test_image_ids, IMG_CHANNELS, X_test)
read_resized_images(MASKS_PATH, test_mask_ids, None, Y_test)

## Input

In [ ]:
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
input = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

In [ ]:
from keras.layers.advanced_activations import LeakyReLU

# activation = LeakyReLU()
activation = 'relu'
# activation = 'tanh'
# activation = 'selu'
kernel_initializer = 'he_normal'
padding = 'same'

## Contraction

The following code was adapted from [here](https://github.com/bnsreenu/python_for_microscopists).

In [ ]:
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(input)
c1 = tf.keras.layers.Dropout(0.2)(c1)
# c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(p1)
c2 = tf.keras.layers.Dropout(0.2)(c2)
# c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
# c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
# c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(p4)
c5 = tf.keras.layers.Dropout(0.2)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(c5)

## Expansion

In [ ]:
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding=padding)(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
# c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding=padding)(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
# c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding=padding)(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(u8)
c8 = tf.keras.layers.Dropout(0.2)(c8)
# c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding=padding)(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(u9)
c9 = tf.keras.layers.Dropout(0.2)(c9)
# c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=activation, kernel_initializer=kernel_initializer, padding=padding)(c9)

## Fully Connected Layer

In [ ]:
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

## Model

In [ ]:
from keras import backend as K
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

## Callbacks

In [ ]:
callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir='tensorboard'),
        tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
        tf.keras.callbacks.ReduceLROnPlateau(),
        tf.keras.callbacks.ModelCheckpoint('models/best_model.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
        ]

## Training
**Note**: Keras splits the validation data as follows: "The validation data is selected from the **last samples** in the x and y data provided, before shuffling."

In [ ]:
!rm -rf "tensorboard"
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=8, epochs=100, callbacks=callbacks)

## Splitting Validation Data
**Note**: Since Keras splits the data for validation from the last samples provided I'm splitting the data in the same manner.

In [ ]:
idx = random.randint(0, len(X_train))

preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

## Setting Threshold

In [ ]:
threshold = 0.5
preds_train_t = (preds_train > threshold).astype(np.uint8)
preds_val_t = (preds_val > threshold).astype(np.uint8)
preds_test_t = (preds_test > threshold).astype(np.uint8)

In [ ]:
# Perform a sanity check on some random training samples
rows = 1
cols = 3
plt.figure(figsize=(16,8))

for i in range(1, rows*cols, 3):
  ix = random.randint(0, len(preds_train_t))
  plt.subplot(rows, cols, i)
  imshow(X_train[ix])
  plt.subplot(rows, cols, i+1)
  imshow(np.squeeze(Y_train[ix]))
  plt.subplot(rows, cols, i+2)
  imshow(np.squeeze(preds_train_t[ix]))

In [ ]:
# Perform a sanity check on some random validation samples
ix = random.randint(0, len(preds_val_t))
plt.figure(figsize=(12,8))
plt.subplot(1, 3, 1)
imshow(X_train[int(X_train.shape[0]*0.9):][ix])
plt.subplot(1, 3, 2)
imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
plt.subplot(1, 3, 3)
imshow(np.squeeze(preds_val_t[ix]))

In [ ]:
# Perform a sanity check on some random test samples
ix = random.randint(0, len(preds_test_t))
plt.figure(figsize=(12,8))
plt.subplot(1, 3, 1)
imshow(X_test[ix])
plt.subplot(1, 3, 2)
imshow(np.squeeze(Y_test[ix]))
plt.subplot(1, 3, 3)
imshow(np.squeeze(preds_test_t[ix]))

In [ ]:
!kill 8819

In [ ]:
# %load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir {"tensorboard"}